In [ ]:
import os
import time
import subprocess

# === CONFIGURATION ===
WATCH_DIR = r"C:\Users\lokes\OneDrive\Documents\Projects\machine_learning-notebooks"
BRANCH = "main"
INTERVAL = 5  # seconds

def git_run(cmd, cwd):
    """Run a Git command and return output."""
    return subprocess.run(cmd, cwd=cwd, capture_output=True, text=True)

def watch_and_sync(directory, branch="main", interval=5):
    print(f"Watching folder: {directory}")
    
    os.chdir(directory)

    # Ensure Git repo exists
    if not os.path.exists(os.path.join(directory, ".git")):
        print("Initializing Git repo...")
        git_run(["git", "init"], directory)
        git_run(["git", "branch", "-M", branch], directory)
        print("Git repo initialized.\n")

    last_snapshot = {}

    while True:
        current_snapshot = {}

        # Scan notebook or python files
        for root, dirs, files in os.walk(directory):
            for filename in files:
                if filename.endswith(".ipynb") or filename.endswith(".py"):
                    filepath = os.path.join(root, filename)
                    current_snapshot[filepath] = os.path.getmtime(filepath)

        # Identify changed files
        changed_files = [
            f for f in current_snapshot
            if f not in last_snapshot or current_snapshot[f] != last_snapshot[f]
        ]

        if changed_files:
            print("Changes detected:", changed_files)

            git_run(["git", "add", "."], directory)
            git_run(["git", "commit", "-m", "Auto-sync commit"], directory)
            git_run(["git", "push", "-u", "origin", branch], directory)

            print("Changes pushed.\n")

        last_snapshot = current_snapshot
        time.sleep(interval)

# Run watcher
watch_and_sync(WATCH_DIR, branch=BRANCH, interval=INTERVAL)
